In [1]:
## 1 we need to download all needed libraries

import pandas as pd
import numpy as np
import yfinance as yf
from scipy.optimize import minimize

In [7]:
## 2 we need to define the data that we want to use, and here we will use yfinance and stocks that were used in the paper
stocks = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'JPM', 'V', 'PG', 'JNJ', 'UNH', 'MA', 'INTC']
start_date = '2019-01-02'
end_date = '2021-12-30'

stock_data = yf.download(stocks, start=start_date, end=end_date)
stock_prices = stock_data['Adj Close']


[*********************100%***********************]  11 of 11 completed


In [14]:
stock_prices = pd.DataFrame(stock_prices)
print(stock_prices.head())

                 AAPL       AMZN      GOOGL       INTC         JNJ        JPM  \
Date                                                                            
2019-01-02  37.994495  76.956497  52.734001  41.343922  113.289452  86.554718   
2019-01-03  34.209961  75.014000  51.273499  39.069481  111.489220  85.324585   
2019-01-04  35.670353  78.769501  53.903500  41.466866  113.360374  88.470131   
2019-01-07  35.590965  81.475502  53.796001  41.660057  112.633194  88.531631   
2019-01-08  36.269440  82.829002  54.268501  41.923512  115.249283  88.364685   

                    MA       MSFT         PG         UNH           V  
Date                                                                  
2019-01-02  185.082962  96.421883  81.445831  228.546417  128.979111  
2019-01-03  176.733063  92.874702  80.874771  222.313919  124.331146  
2019-01-04  185.102463  97.194229  82.525444  224.913895  129.687500  
2019-01-07  186.526627  97.318199  82.195335  225.345688  132.026031  
2019-0

In [10]:
## 3 now we calculate the return but it will be daily as it is in the paper. For NaN values it is better to use 
##ffill() the last value,  instead of dropna()
returns = stock_prices.pct_change().ffill()


In [11]:
print(returns)

                AAPL      AMZN     GOOGL      INTC       JNJ       JPM  \
Date                                                                     
2019-01-02       NaN       NaN       NaN       NaN       NaN       NaN   
2019-01-03 -0.099607 -0.025241 -0.027696 -0.055013 -0.015891 -0.014212   
2019-01-04  0.042689  0.050064  0.051294  0.061362  0.016783  0.036866   
2019-01-07 -0.002226  0.034353 -0.001994  0.004659 -0.006415  0.000695   
2019-01-08  0.019063  0.016612  0.008783  0.006324  0.023227 -0.001886   
...              ...       ...       ...       ...       ...       ...   
2021-12-22  0.015319  0.003638  0.020509  0.003939  0.004306  0.003908   
2021-12-23  0.003644  0.000184  0.003425  0.006671  0.001906  0.003574   
2021-12-27  0.022975 -0.008178  0.006738  0.012278  0.008440  0.005723   
2021-12-28 -0.005767  0.005844 -0.008245 -0.003466  0.004008  0.003035   
2021-12-29  0.000502 -0.008555 -0.000218  0.001353  0.007044 -0.000504   

                  MA      MSFT       

In [12]:
## 4 Mean Variance Optimization
def portfolio_variance(weights, cov_matrix):
    return np.dot(weights.T, np.dot(cov_matrix, weights))

def objective_function(weights, returns, cov_matrix, risk_free_rate):
    portfolio_return = np.dot(weights.T, returns.mean())
    portfolio_std = np.sqrt(portfolio_variance(weights, cov_matrix))
    sharpe_ratio = (portfolio_return - risk_free_rate) / portfolio_std
    return -sharpe_ratio  # Negate for minimization

def optimize_portfolio(returns, cov_matrix, risk_free_rate):
    num_assets = len(returns.columns)
    initial_weights = np.ones(num_assets) / num_assets
    bounds = [(0, 1)] * num_assets
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    
    result = minimize(
        objective_function,
        initial_weights,
        args=(returns, cov_matrix, risk_free_rate),
        method='SLSQP',
        bounds=bounds,
        constraints=constraints
    )
    
    if result.success:
        return result.x
    else:
        raise ValueError(result.message)

cov_matrix = returns.cov()
risk_free_rate = 0.02  # Example risk-free rate (2%)
weights_msrp = optimize_portfolio(returns, cov_matrix, risk_free_rate)

In [17]:
## 5 Backtesting

# Calculate portfolio cumulative returns
portfolio_cumulative_returns = (1 + portfolio_returns).cumprod()

# Calculate maximum drawdown
previous_peaks = np.maximum.accumulate(portfolio_cumulative_returns)
drawdown = (portfolio_cumulative_returns / previous_peaks) - 1
maximum_drawdown = drawdown.min()

# Print the maximum drawdown
print("Maximum Drawdown: {:.2%}".format(maximum_drawdown))

Maximum Drawdown: nan%
